**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pick.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20201015', drake_build='nightly')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydrake.all
import os
from IPython.display import display

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw
import pydrake.multibody.jupyter_widgets
from manipulation.utils import FindResource
from manipulation.jupyter_widgets import MakeJointSlidersThatPublishOnCallback

In [ ]:
def AddPlanarBinAndCracker(plant):
    parser = pydrake.multibody.parsing.Parser(plant)
    bin = parser.AddModelFromFile(FindResource("models/planar_bin.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base", bin), RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [0,0,-0.015]))

    planar_joint_frame = plant.AddFrame(pydrake.multibody.tree.FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    box = parser.AddModelFromFile(pydrake.common.FindResourceOrThrow("drake/manipulation/models/ycb/sdf/003_cracker_box.sdf"))
    box_frame = plant.AddFrame(pydrake.multibody.tree.FixedOffsetFrame("box_frame", plant.GetFrameByName("base_link_cracker", box), 
                                   RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    box_joint = plant.AddJoint(pydrake.multibody.tree.PlanarJoint("box_joint", planar_joint_frame, box_frame))
    box_joint.set_default_translation([0, 0.033400])
    return box


def AddPlanarGripper(plant):
    parser = pydrake.multibody.parsing.Parser(plant)
    parser.package_map().Add("wsg_50_description", os.path.dirname(pydrake.common.FindResourceOrThrow("drake/manipulation/models/wsg_50_description/package.xml")))
    gripper = parser.AddModelFromFile(FindResource("models/schunk_wsg_50_welded_fingers.sdf"), "gripper")
    gripper_body = plant.GetBodyByName("body", gripper)

    # Add a planar joint the old fashioned way (so that I can have three actuators):
    gripper_false_body1 = plant.AddRigidBody("false_body1", gripper, pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    gripper_false_body2 = plant.AddRigidBody("false_body2", gripper, pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    gripper_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("gripper_x", plant.world_frame(), plant.GetFrameByName("false_body1"), [1, 0, 0], -.3, .3))
    plant.AddJointActuator("gripper_x", gripper_x)
    gripper_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("gripper_z", plant.GetFrameByName("false_body1"), plant.GetFrameByName("false_body2"), [0, 0, 1], 0.0, 0.5))
    gripper_z.set_default_translation(0.25)
    plant.AddJointActuator("gripper_z", gripper_z)
    gripper_frame = plant.AddFrame(pydrake.multibody.tree.FixedOffsetFrame("gripper_frame", plant.GetFrameByName("body", gripper), 
                                    RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    gripper_theta = plant.AddJoint(pydrake.multibody.tree.RevoluteJoint("gripper_theta", plant.GetFrameByName("false_body2"), gripper_frame, [0, -1, 0],  -np.pi, np.pi))
    plant.AddJointActuator("gripper_theta", gripper_theta)

    return gripper

def MakePlanarGripperOnlyPlant():
    plant = pydrake.multibody.plant.MultibodyPlant(time_step=0.005)
    AddPlanarGripper(plant)
    plant.Finalize()
    return plant

def AddPointFinger(plant):
    mu = 1.0
    finger = plant.AddModelInstance("finger")
    false_body1 = plant.AddRigidBody("false_body1", finger, pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    finger_body = plant.AddRigidBody("body", finger, pydrake.multibody.tree.SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))
    shape = pydrake.geometry.Sphere(0.01)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_x", plant.world_frame(), plant.GetFrameByName("false_body1"), [1, 0, 0], -.3, .3))
    plant.AddJointActuator("finger_x", finger_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_z", plant.GetFrameByName("false_body1"), plant.GetFrameByName("body"), [0, 0, 1], 0.0, 0.5))
    finger_z.set_default_translation(0.25)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

def MakeFingerOnlyPlant():
    plant = pydrake.multibody.plant.MultibodyPlant(time_step=0.005)
    AddPointFinger(plant)
    plant.Finalize()
    return plant

# Commanding a constant force

In [ ]:
import matplotlib.pyplot as plt

class PointFingerForceControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorInputPort(
            "desired_contact_force", pydrake.systems.framework.BasicVector(2))
        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]

        desired_force = self.get_input_port(0).Eval(context)
        output.SetFromVector(-finger_mass*g - desired_force)

def force_control_point_finger(desired_horizontal_forces=[4.0], duration=1.0, draw=False):
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    box = AddPlanarBinAndCracker(plant)
    finger = AddPointFinger(plant)
    plant.GetJointByName("finger_x").set_default_translation(0.15)
    plant.GetJointByName("finger_z").set_default_translation(0.05)
    plant.Finalize()

    if draw:
        vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph, delete_prefix_on_load=False)
        vis.vis['drake']['4'].delete()  # Only flush the gripper geometry.
    #    vis.set_planar_viewpoint(xmin=-.5, xmax=.5, ymin=-0.1, ymax=0.6)
        contact_visualizer = builder.AddSystem(pydrake.systems.meshcat_visualizer.MeshcatContactVisualizer(vis, plant=plant, contact_force_radius=0.005))
        builder.Connect(plant.get_contact_results_output_port(),
                        contact_visualizer.GetInputPort("contact_results"))

    controller = builder.AddSystem(PointFingerForceControl(plant))
    builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())
    builder.ExportInput(controller.get_input_port(), "desired_contact_force")

    log = pydrake.systems.primitives.LogOutput(plant.get_state_output_port(finger), builder)

    diagram = builder.Build()

    plt.figure()
    lines = []
    for f in desired_horizontal_forces:
        log.reset()
        simulator = pydrake.systems.analysis.Simulator(diagram)
        context = simulator.get_mutable_context()
        diagram.get_input_port().FixValue(context, [f, 0])

        simulator.AdvanceTo(duration)
        t = log.sample_times()
        x = log.data()
        lines.append(plt.plot(t, x[0, :])[0])

    plt.legend(lines, [("$f^{c_d}$ = " + str(f)) for f in desired_horizontal_forces])
    plt.xlabel('t (sec)')
    plt.ylabel('x (m)')

force_control_point_finger([0.25, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0])


# Teleop version

Picking up that box is not easy!  Give it a shot.  Just see if you can get the box to be inside the fingers (so that if we *did* close the fingers we could grasp the box).


In [ ]:
def teleop_box_pickup(point_finger=True):
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    AddPlanarBinAndCracker(plant)
    gripper = AddPointFinger(plant) if point_finger else AddPlanarGripper(plant)
    plant.Finalize()

    vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph, delete_prefix_on_load=False)
    vis.vis['drake']['4'].delete()  # Only flush the gripper geometry.
#    vis.set_planar_viewpoint(xmin=-.5, xmax=.5, ymin=-0.1, ymax=0.6)
    contact_visualizer = builder.AddSystem(pydrake.systems.meshcat_visualizer.MeshcatContactVisualizer(vis, plant=plant, contact_force_radius=0.005))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    controller_plant = MakeFingerOnlyPlant() if point_finger else MakePlanarGripperOnlyPlant()
    N = controller_plant.num_positions()
    kp = [100]*N
    ki = [1]*N
    kd = [2*np.sqrt(kp[0])]*N
    controller = builder.AddSystem(pydrake.systems.controllers.InverseDynamicsController(controller_plant, kp, ki, kd, False))
    builder.Connect(plant.get_state_output_port(gripper), controller.get_input_port_estimated_state())

    display(ipywidgets.Label(value="Set desired positions:"))
    position_sliders = builder.AddSystem(pydrake.multibody.jupyter_widgets.JointSliders(controller_plant))
    positions_to_state = builder.AddSystem(pydrake.systems.primitives.Multiplexer([N, N]))
    builder.Connect(position_sliders.get_output_port(), positions_to_state.get_input_port(0))
    zeros = builder.AddSystem(pydrake.systems.primitives.ConstantVectorSource([0]*N))
    builder.Connect(zeros.get_output_port(0), positions_to_state.get_input_port(1))
    builder.Connect(positions_to_state.get_output_port(), controller.get_input_port_desired_state())

    display(ipywidgets.Label(value="Set feedforward forces/torques:"))
    force_sliders = builder.AddSystem(pydrake.multibody.jupyter_widgets.JointSliders(controller_plant))
    for slider in force_sliders._slider:
        slider.min = -1.0
        slider.max = 1.0
        slider.value = 0.0

    adder = builder.AddSystem(pydrake.systems.primitives.Adder(2, N))
    builder.Connect(controller.get_output_port(), adder.get_input_port(0))
    builder.Connect(force_sliders.get_output_port(), adder.get_input_port(1))
    builder.Connect(adder.get_output_port(), plant.get_actuation_input_port())

    diagram = builder.Build()
    simulator = pydrake.systems.analysis.Simulator(diagram)

    if running_as_notebook:
        stop_button = ipywidgets.ToggleButton(value=False, description='Stop Simulation')
        display(stop_button)
        while not stop_button.value:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        stop_button.value = False
    else:
        simulator.AdvanceTo(0.01)

teleop_box_pickup(point_finger=False)

#  A simple solution with force control

In [ ]:
def force_control_point_finger():
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    box = AddPlanarBinAndCracker(plant)
    finger = AddPointFinger(plant)
    plant.GetJointByName("finger_x").set_default_translation(0.15)
    plant.GetJointByName("finger_z").set_default_translation(0.05)
    plant.Finalize()

    vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph, delete_prefix_on_load=False)
    vis.vis['drake']['4'].delete()  # Only flush the gripper geometry.
#    vis.set_planar_viewpoint(xmin=-.5, xmax=.5, ymin=-0.1, ymax=0.6)
    contact_visualizer = builder.AddSystem(pydrake.systems.meshcat_visualizer.MeshcatContactVisualizer(vis, plant=plant, contact_force_radius=0.005))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    class ForceController(pydrake.systems.framework.LeafSystem):

        def __init__(self, plant):
            pydrake.systems.framework.LeafSystem.__init__(self)
            self._plant = plant

            self.DeclareVectorInputPort(
                "box_state", pydrake.systems.framework.BasicVector(6))
            self.DeclareVectorInputPort(
                "finger_state", pydrake.systems.framework.BasicVector(4))
            self.DeclareVectorOutputPort(
                "finger_force", pydrake.systems.framework.BasicVector(2), 
                                        self.CalcOutput)

        def CalcOutput(self, context, output):
            finger_mass = 1
            box_mass = 0.411  # Let's revisit whether we need this!
            mu_A = 1.0
            mu_C = 1.0
            # assume I only have a fraction of the friction, to accommodate other approximations.
            mu_A *= 0.5
            mu_C *= 0.5
            g = self._plant.gravity_field().gravity_vector()[[0,2]]

            if context.get_time() < 1.0:
                u = np.array([-4.0, 0.0])
                output.SetFromVector(u - finger_mass*g)
                return

            box_state = self.get_input_port(0).Eval(context)
            theta = box_state[2]
            theta_dot = box_state[5]
            R_WC = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
            R_CW = R_WC.T
            finger_state = self.get_input_port(1).Eval(context)

            prog = pydrake.solvers.mathematicalprogram.MathematicalProgram()
            f_WA = prog.NewContinuousVariables(2, "f_WA")  # force from wall to box
            f_WC = prog.NewContinuousVariables(2, "f_WC")  # force from finger to box
            f_CC = np.matmul(R_CW, f_WC)

            prog.AddConstraint(f_WA[0] >= 0)
            prog.AddConstraint(f_WC[0] <= 0)

            # Friction cone at A (normal is aligned with x)
            prog.AddConstraint(f_WA[1] <= mu_A*f_WA[0])
            prog.AddConstraint(f_WA[1] >= -mu_A*f_WA[0])

            # Friction cone at C (normal is aligned with -x)
            prog.AddConstraint(f_CC[1] <= -mu_C*f_CC[0])
            prog.AddConstraint(f_CC[1] >= mu_C*f_CC[0])

            # Horizontal forces are equal and opposite
            prog.AddConstraint(f_WA[0] == -f_WC[0])

            # vertical forces can hold the mass
            prog.AddConstraint(f_WA[1] + f_WC[1] >= -box_mass*g[1])
            
            # PID control on f_CC[1]
            kp = 100.0
            kd = 2.0*np.sqrt(kp)  # should really be kp*I on inside
            theta_desired = 0.5*(context.get_time() - 1.0)
            f_CC_Cx_desired = kp*(theta_desired - theta) - kd*theta_dot
            prog.AddCost((f_CC_Cx_desired - f_CC[1])**2)
            result = pydrake.solvers.mathematicalprogram.Solve(prog)
            
            if not result.is_success():
#                print(result.GetInfeasibleConstraintNames(prog))
                output.SetFromVector(-finger_mass*g)
                return
            
            f_WC = result.GetSolution(f_WC)
#            print(f_WC)
            output.SetFromVector(f_WC - finger_mass*g)


    controller = builder.AddSystem(ForceController(plant))
    builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())
    builder.Connect(plant.get_state_output_port(box), controller.get_input_port(0))
    builder.Connect(plant.get_state_output_port(finger), controller.get_input_port(1))

    diagram = builder.Build()
    if False: # True:  # Useful for debugging the initial setup
        context = diagram.CreateDefaultContext()
        vis.load()
        MakeJointSlidersThatPublishOnCallback(plant, diagram, context)
        return

    simulator = pydrake.systems.analysis.Simulator(diagram)

    if running_as_notebook:
        simulator.set_target_realtime_rate(.2)
        vis.start_recording()
        simulator.AdvanceTo(3.5)
        vis.stop_recording()
        vis.vis['drake/contact_forces'].delete()  # TODO: make my recording include the contact forces!
        vis.publish_recording()
    else:
        simulator.AdvanceTo(0.01)


force_control_point_finger()